# 02 London Borough Accessibilty To OS Open Greenspace 
---

Quanitfying London Borough-accessibilty to a subset of OS Open Greenspace.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

###  Create GeoDataFrame from GeoPackage (GPKG)

In [ ]:
# Create a GeoPandas GeoDataFrame from a GeoPackage (GPKG)
osogs = gpd.read_file(
    filename="../../data/ordnance-survey/os-open-greenspace-gb.gpkg",
    # GPKG layer
    layer="greenspace_site",
)

###  Spatially subset GeoDataFrame

Using coordinate-based indexer to spatially subset by bounding box (BBOX).

In [ ]:
# Coordinate-based indexer to select by intersection with BBOX
# Greater London BBOX
osogs_filtered = osogs.cx[
    503568.1996:561957.4962, 155850.7975:200933.9026
]  # xmin:xmax, ymin:ymax

In [ ]:
# GeoDataFrame row count
osogs_filtered.shape[0]

In [ ]:
osogs_filtered.head()

### Subset GeoDataFrame by function

Filter OS Open Greenspace retaining only features where the function column is equal to 'Playing Field' or 'Public Park And Garden'.

In [ ]:
# Subset GeoDataFrame by function
osogs_filtered = osogs_filtered.loc[
    (osogs_filtered["function"].isin(["Playing Field", "Public Park And Garden"])),
    ["id", "function", "geometry"],
]

In [ ]:
# GeoDataFrame row count
osogs_filtered.shape[0]

In [ ]:
osogs_filtered.head()

In [ ]:
# Create a GeoPandas GeoDataFrame from a GeoPackage (GPKG)
lad = gpd.read_file(
    filename="../../data/office-for-national-statistics/local-authority-districts-dec-2021-gb-bfc.gpkg",
    # GPKG layer
    layer="LAD_DEC_2021_GB_BFC",
)

In [ ]:
# List columns
lad.columns

In [ ]:
# Head of GeoDataFrame
lad.head()

In [ ]:
# Row count
lad.shape[0]

In [ ]:
# Create figure and axes objects
f, ax = plt.subplots(figsize=(5, 10))

# Set title
ax.set_title("Local Authority District (LAD) Boundaries 2021")
# Turn axis off
ax.set_axis_off()

# Plot GeoDataFrame
lad.plot(ax=ax, color="#af58ba");

In [ ]:
# Rename GeoDataFrame columns
lad.rename(columns={"LAD21CD": "lad21cd", "LAD21NM": "lad21nm"}, inplace=True)

In [ ]:
# Subset GeoDataFrame columns
lad = lad[["lad21cd", "lad21nm", "geometry"]]

In [ ]:
lad.head()

In [ ]:
# Count rows by geometry type
lad["geometry"].geom_type.value_counts()

In [ ]:
# Count rows by validity
lad["geometry"].is_valid.value_counts()

### Spatial join 

Compute the intersection (shared geometry) between OS Open Greenspace and Local Authority District boundaries.

In [ ]:
# Join OS Open Greenspace to LAD boundaries
osogs_filtered2lad = osogs_filtered.overlay(lad, how="intersection")

# Head of GeoDataFrame
osogs_filtered2lad.head()

In [ ]:
# Whats going on...
osogs_filtered2lad.groupby("id")\
.agg(green_space_id_count =('id','count'))\
.reset_index(drop = False)\
.sort_values(by = 'green_space_id_count', ascending = False)

In [ ]:
osogs_filtered2lad[osogs_filtered2lad['id'] == '0295ED34-679C-5C37-E063-AAEFA00A445E']#.plot(column = 'lad21cd')

### Add `greenspace_area` column

In [ ]:
osogs_filtered2lad["greenspace_area"] = osogs_filtered2lad["geometry"].area

In [ ]:
# List columns
osogs_filtered2lad.head()

### Sum areas by LAD

In [ ]:
osogs_filtered2lad_grouped = (
    osogs_filtered2lad.groupby(["lad21cd", "lad21nm"])["greenspace_area"]
    .sum()
    .reset_index()
)

osogs_filtered2lad_grouped.head()

In [ ]:
len(osogs_filtered2lad_grouped)

### Create DataFrame from Excel

Read the latest Office for National Statistics (ONS) mid-year population estimates for Local Authority District (LAD) boundaries.

In [ ]:
# Create Pandas DataFrame from Excel
pop_est = pd.read_excel(
    io="../../data/office-for-national-statistics/uk-population-estimates-mid-2021.xls",
    sheet_name="MYE2 - Persons",
    skiprows=7,
)

# Return head of DataFrame
pop_est.head()

In [ ]:
# List DataFrame columns
pop_est.columns

In [ ]:
# Count rows by Geography classes
pop_est["Geography"].value_counts()

In [ ]:
# Subset rows where Geography column is equal to 'London Borough'
pop_est = pop_est.loc[
    (pop_est["Geography"] == "London Borough"), ["Code", "Name", "All ages"]
]

# Rename columns
pop_est.rename(
    columns={"Code": "lad21cd", "Name": "lad21nm", "All ages": "population"},
    inplace=True,
)

In [ ]:
# Return head of DataFrame
pop_est.head()

In [ ]:
lad.head()

In [ ]:
osogs_pop_est_lad_merged = (
    # Start with the geometries we want to join to
    lad
    # Drop the lad21nm because we already have that in the populations dataframe
    .drop(columns=["lad21nm"])
    # Join the population data
    .merge(pop_est, on="lad21cd", how="right")
    # Join to the aggregated greenspace areas
    .merge(
        osogs_filtered2lad_grouped.drop(columns=["lad21nm"]), on="lad21cd", how="left"
    )
    # Any London boroughs without a greenspace_area value has no greenspace, so fill in the value
    .fillna({"greenspace_area": 0})
)[["lad21cd", "lad21nm", "greenspace_area", "population", "geometry"]]

osogs_pop_est_lad_merged.head()

In [ ]:
# Calculate OS Open Greenspace area per LAD per head
osogs_pop_est_lad_merged["area2population"] = (
    osogs_pop_est_lad_merged["greenspace_area"] / osogs_pop_est_lad_merged["population"]
)

In [ ]:
# List DataFrame columns
osogs_pop_est_lad_merged.columns

In [ ]:
# Create figure and axes objects
f, ax = plt.subplots(figsize=(8, 8))

# Turn axis off
ax.axis("off")

# Plot data
osogs_pop_est_lad_merged.plot(
    ax=ax,
    column="area2population",
    legend=True,
    legend_kwds={"shrink": 0.4, "label": r"$m^2$ of greenspace per person"},
)

In [ ]:
# Write to Geopackage (GPKG) database file
osogs_pop_est_lad_merged.to_file(
    filename="../../data/lad-os-open-greenspace-area-per-head.gpkg", driver="GPKG"
)